# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and appendding it        
        for line in csvreader:
            full_data_rows_list.append(line) 
        
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra coding portion of project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance in local machine 
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Creating Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS eventdb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('eventdb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Creating queries to ask the following three questions of the data

### 1. Getting the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Getting only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Getting artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS session_library "
query1 = query1 + "(session_id int, item_in_Session int, artist text,song_title text, length float, PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [9]:
## inserting values from csv files
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO session_library (session_id,item_in_session,artist,song_title,length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Adding in the SELECT statement to verify the data was entered into the table
s_query = "SELECT artist,song_title,length FROM session_library WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(s_query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Now repeating the steps for query 2:

In [11]:
## Query 2: Getting name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# Creating table user_library:
query2 = "CREATE TABLE IF NOT EXISTS user_library "
query2 = query2 + "(user_id int,session_id int, item_in_Session int, artist text,song_title text, user_name text, PRIMARY KEY (user_id,session_id,item_in_session))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

# Inserting values
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (user_id,session_id,item_in_session,artist,song_title,user_name)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9],line[1]+' '+line[4]))

# Verifying Results through SELECT statement:
s_query = 'SELECT artist,song_title,user_name FROM user_library WHERE user_id = 10 AND session_id = 182'
try:
    rows=session.execute(s_query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist,row.song_title,row.user_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Now repeating the steps again for query 3:

In [12]:
## Query 3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Creating table song_library
query3 = "CREATE TABLE IF NOT EXISTS song_library "
query3 = query3 + "(song_title text,user_id int, first_name text, last_name text, PRIMARY KEY (song_title,user_id))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

# Inserting values into table song_library:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (song_title,user_id,first_name,last_name)"
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

# verifying results through SELECT statement:
s_query = "SELECT first_name,last_name FROM song_library WHERE song_title = 'All Hands Against His Own'"
try:
    rows=session.execute(s_query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name,row.last_name)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [13]:
## Dropping the table before closing out the sessions

In [14]:
try:
    session.execute('DROP TABLE IF EXISTS session_library')
except Exception as e:
    print(e)
try:
    session.execute('DROP TABLE IF EXISTS user_library')
except Exception as e:
    print(e)
try:
    session.execute('DROP TABLE IF EXISTS song_library')
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()